In [1]:
import transformers
from datasets import load_dataset, load_metric
import pandas as pd

c:\Users\love4\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
news = load_dataset("csv", data_files="./data/news_collection_new.csv")

Using custom data configuration default-e640655710a3e75d
Found cached dataset csv (C:/Users/love4/.cache/huggingface/datasets/csv/default-e640655710a3e75d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 427.03it/s]


In [3]:
datasets_train_validation = news["train"].train_test_split(test_size=1000)

news["train"] = datasets_train_validation["train"].shuffle()
news["validation"] = datasets_train_validation["test"].shuffle()

In [4]:
import nltk
nltk.download('punkt')
import string

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("yihsuan/mt5_chinese_small")

model = AutoModelForSeq2SeqLM.from_pretrained("yihsuan/mt5_chinese_small")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\love4\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
prefix = "summarize: "
max_input_length = 512
max_target_length = 64


def preprocess_data(examples):
  
  inputs = [prefix + text for text in examples["desc"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [6]:
tokenized_datasets = news.map(preprocess_data, batched=True)

  0%|          | 0/47 [00:00<?, ?ba/s]c:\Users\love4\anaconda3\envs\pytorch\lib\site-packages\transformers\tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
  0%|          | 0/1 [00:00<?, ?ba/s]


In [7]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [8]:
batch_size = 4
model_name = "mt5-base-news-title-generation"
model_dir = f"./Models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps = 500,
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=4e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [10]:
import evaluate
metric = evaluate.load("rouge")

In [11]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [12]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained("yihsuan/mt5_chinese_small")

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at C:\Users\love4/.cache\huggingface\hub\models--yihsuan--mt5_chinese_small\snapshots\338c78a9a4e6ff81d4e613a7b742a99b864b8f28\config.json
Model config MT5Config {
  "_name_or_path": "yihsuan/mt5_chinese_small",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 250100
}

loadi

In [13]:
%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

Reusing TensorBoard on port 6006 (pid 9564), started 4:07:42 ago. (Use '!kill 9564' to kill it.)

In [14]:
trainer.train()

loading configuration file config.json from cache at C:\Users\love4/.cache\huggingface\hub\models--yihsuan--mt5_chinese_small\snapshots\338c78a9a4e6ff81d4e613a7b742a99b864b8f28\config.json
Model config MT5Config {
  "_name_or_path": "yihsuan/mt5_chinese_small",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 250100
}

loadi

{'loss': 3.3158, 'learning_rate': 0.00038842588581824484, 'epoch': 0.04}


                                                   
  4%|▍         | 501/11543 [01:26<23:49:36,  7.77s/it]

{'eval_loss': nan, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_rougeLsum': 0.0, 'eval_gen_len': 0.0, 'eval_runtime': 43.5651, 'eval_samples_per_second': 22.954, 'eval_steps_per_second': 5.739, 'epoch': 0.04}


  9%|▊         | 1000/11543 [02:04<13:34, 12.94it/s]  The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: url, image, title, date, source, desc. If url, image, title, date, source, desc are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4


...

{'eval_loss': nan, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_rougeLsum': 0.0, 'eval_gen_len': 0.0, 'eval_runtime': 46.4022, 'eval_samples_per_second': 21.551, 'eval_steps_per_second': 5.388, 'epoch': 0.78}


Model weights saved in ./Models/mt5-base-news-title-generation\checkpoint-9000\pytorch_model.bin
tokenizer config file saved in ./Models/mt5-base-news-title-generation\checkpoint-9000\tokenizer_config.json
Special tokens file saved in ./Models/mt5-base-news-title-generation\checkpoint-9000\special_tokens_map.json
Copy vocab file to ./Models/mt5-base-news-title-generation\checkpoint-9000\spiece.model
Deleting older checkpoint [Models\mt5-base-news-title-generation\checkpoint-7000] due to args.save_total_limit
 82%|████████▏ | 9500/11543 [25:50<02:35, 13.10it/s]  The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: url, image, title, date, source, desc. If url, image, title, date, source, desc are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4


{'loss': 0.0, 'learning_rate': 7.654855756735684e-05, 'epoch': 0.82}


 82%|████████▏ | 9500/11543 [26:05<02:35, 13.10it/s]

KeyboardInterrupt: 